In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [16]:
data = pd.read_csv('normalized_mustard_dataset.csv')
data.head()

,utterance,speaker,context,context_speakers,show,sarcasm
0,It's just a privilege to watch your mind at work.,SHELDON,['I never would have identified the fingerprin...,"['LEONARD', 'SHELDON']",BBT,True
1,I don't think I'll be able to stop thinking ab...,PENNY,['This is one of my favorite places to kick ba...,"['HOWARD', 'PENNY', 'HOWARD', 'HOWARD', 'HOWAR...",BBT,True
2,"Since it's not bee season, you can have my epi...",SHELDON,"['Here we go. Pad thai, no peanuts.', 'But doe...","['LEONARD', 'HOWARD', 'LEONARD']",BBT,False
3,"Lois Lane is falling, accelerating at an initi...",SHELDON,['A marathon? How many Superman movies are the...,"['PENNY', 'SHELDON', 'PENNY', 'SHELDON', 'SHEL...",BBT,False
4,I'm just inferring this is a couch because the...,SHELDON,"[""Great Caesar's ghost, look at this place."", ...","['SHELDON', 'LEONARD', 'SHELDON', 'SHELDON', '...",BBT,True


In [22]:
vocab_size = 10000
embedding_size = 16
required_data = data[['utterance', 'sarcasm']]

utterances = np.array(required_data['utterance'])
utterance_list = utterances.tolist()

sarcasm_states = np.array(required_data['sarcasm'])

tfidf = TfidfVectorizer()
X1 = tfidf.fit_transform(utterance_list)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   utterance         690 non-null    object
 1   speaker           690 non-null    object
 2   context           690 non-null    object
 3   context_speakers  690 non-null    object
 4   show              690 non-null    object
 5   sarcasm           690 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 27.8+ KB


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X1,sarcasm_states, test_size=0.25)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_size, input_length=1977),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(x_train, np.array(y_train), epochs=10, validation_data=(x_test, np.array(y_test)))

In [28]:
user_statement = input("Tell me something sarcastic: \n")
data = tfidf.transform([user_statement]).toarray()
output = model.predict(data)
print(output.flatten())

<class 'numpy.ndarray'>
[0.49690127]
